In [1]:
import pyterrier as pt
pt.init()
from pyterrier.measures import *
from pyterrier_adaptive import GAR, CorpusGraph
from pyterrier_t5 import MonoT5ReRanker
from pyterrier_pisa import PisaIndex

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
# Create required components
dataset = pt.get_dataset('irds:msmarco-passage')
retriever = PisaIndex.from_dataset('msmarco_passage').bm25()
scorer = pt.text.get_text(dataset, 'text') >> MonoT5ReRanker(verbose=False, batch_size=16)
graph = CorpusGraph.from_dataset('msmarco_passage', 'corpusgraph_bm25_k16').to_limit_k(8)

In [3]:
# A simple example
pipeline = retriever >> GAR(scorer, graph) >> pt.text.get_text(dataset, 'text')

pipeline.search('clustering hypothesis information retrieval')

,qid,query,docno,rank,score,iteration,text
0,1,clustering hypothesis information retrieval,2180710,0,-0.017059,0,Cluster analysis or clustering is the task of ...
1,1,clustering hypothesis information retrieval,8430269,1,-0.166563,1,Clustering is the grouping of a particular set...
2,1,clustering hypothesis information retrieval,1091429,2,-0.208345,1,Clustering is a fundamental data analysis meth...
3,1,clustering hypothesis information retrieval,2180711,3,-0.341018,5,Cluster analysis or clustering is the task of ...
4,1,clustering hypothesis information retrieval,6031959,4,-0.367014,5,Cluster analysis or clustering is the task of ...
...,...,...,...,...,...,...,...
995,1,clustering hypothesis information retrieval,2471954,995,-14.281301,33,Now we will calculate the distance to the loca...
996,1,clustering hypothesis information retrieval,5237256,996,-14.314410,22,Retrieve null date from table. 1 Re: Retrieve...
997,1,clustering hypothesis information retrieval,8058446,997,-14.336508,31,Modified Newtonian dynamics - Observational ev...
998,1,clustering hypothesis information retrieval,8828444,998,-14.382571,41,"Calculate mean/avaerge, range, median and mode..."


In [4]:
dataset = pt.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')
pt.Experiment(
    [retriever, retriever >> scorer, retriever >> GAR(scorer, graph)],
    dataset.get_topics(),
    dataset.get_qrels(),
    [nDCG, MAP(rel=2), R(rel=2)@1000],
    names=['bm25', 'bm25 >> monot5', 'bm25 >> GAR(monot5)']
)

,name,nDCG,AP(rel=2),R(rel=2)@1000
0,bm25,0.602325,0.303099,0.755495
1,bm25 >> monot5,0.696266,0.481454,0.755495
2,bm25 >> GAR(monot5),0.724501,0.489978,0.825952
